In [1]:
from utils2 import DataLoaderSegmentation
import torch
import torchvision
from torchvision import datasets, transforms
import segmentation_models_pytorch as smp
import os
import matplotlib.pyplot as plt
import albumentations as albu
import torch.utils.data as data_utils
from segmentation_models_pytorch import utils


In [ ]:
 "C:/Users/rasic/OpenCV/DL_WITH_KERAS/QuPath - Copy for DL/tiles/images/"

In [ ]:
TRAIN_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/train/"
VALID_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/valid/"
TEST_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/test/"

In [2]:
TRAIN_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/train/"
VALID_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/valid/"
TEST_PATH = "E:/Niklas_projekt/LVSegmentationCode/dataset/test/"

train_image_files = [TRAIN_PATH + "/images/" + file for file in os.listdir(TRAIN_PATH + "/images/")]
train_mask_files = [TRAIN_PATH + "/masks/" + file for file in os.listdir(TRAIN_PATH + "/masks/")]

valid_image_files = [VALID_PATH + "/images/" + file for file in os.listdir(VALID_PATH + "/images/")]
valid_mask_files = [VALID_PATH + "/masks/" + file for file in os.listdir(VALID_PATH + "/masks/")]

test_image_files = [TEST_PATH + "/images/" + file for file in os.listdir(TEST_PATH + "/images/")]
test_mask_files = [TEST_PATH + "/masks/" + file for file in os.listdir(TEST_PATH + "/masks/")]





In [3]:
model = smp.DeepLabV3(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
         # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [4]:
preprocessing_fn = smp.encoders.get_preprocessing_fn("resnet34", "imagenet")

In [5]:
preprocessing_fn

functools.partial(<function preprocess_input at 0x0000021E7FE060E0>, input_space='RGB', input_range=[0, 1], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [3]:
augmentations = transforms.Compose([transforms.RandomRotation(degrees=(20)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor()])


In [4]:
augmentations_mask = transforms.Compose([transforms.RandomRotation(degrees=(20)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor()])

In [5]:
transform_to_tensor_and_normalize = transforms.Compose([transforms.ToTensor()])
transforms_to_tensor = transforms.Compose([transforms.ToTensor()])

In [6]:
train_dataset = DataLoaderSegmentation(img_files=train_image_files, mask_files=train_mask_files, transforms_img=augmentations, transforms_mask=augmentations)
valid_dataset = DataLoaderSegmentation(img_files=valid_image_files, mask_files=valid_mask_files, transforms_img=transform_to_tensor_and_normalize, transforms_mask=transforms_to_tensor)
test_dataset = DataLoaderSegmentation(img_files=test_image_files, mask_files=test_mask_files, transforms_img=transform_to_tensor_and_normalize, transforms_mask=transforms_to_tensor)

In [7]:
img, mask = train_dataset[0]

In [8]:
img.shape

torch.Size([3, 128, 128])

In [9]:
train_loader =  data_utils.DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4)
valid_loader = data_utils.DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

In [10]:
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [11]:
loss = smp.utils.losses.JaccardLoss()
metrics = [
    smp.utils.metrics.IoU(),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.01),
])

In [12]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device="cuda",
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device="cuda",
    verbose=True,
)

In [13]:
max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')


Epoch: 0
valid: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s, jaccard_loss - -5.015, iou_score - 0.1552]
Model saved!

Epoch: 1
valid: 100%|██████████| 4/4 [00:03<00:00,  1.23it/s, jaccard_loss - -95.87, iou_score - 0.003041] 

Epoch: 2
valid: 100%|██████████| 4/4 [00:02<00:00,  1.36it/s, jaccard_loss - -1.103, iou_score - 0.1331]

Epoch: 3
valid: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s, jaccard_loss - -17.13, iou_score - 0.3873]
Model saved!

Epoch: 4
valid: 100%|██████████| 4/4 [00:03<00:00,  1.32it/s, jaccard_loss - -27.67, iou_score - 0.6224]
Model saved!

Epoch: 5
valid: 100%|██████████| 4/4 [00:03<00:00,  1.27it/s, jaccard_loss - -30.55, iou_score - 0.6511]
Model saved!

Epoch: 6
valid: 100%|██████████| 4/4 [00:02<00:00,  1.34it/s, jaccard_loss - -46.78, iou_score - 0.6182]

Epoch: 7
valid: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s, jaccard_loss - -26.19, iou_score - 0.9042]
Model saved!

Epoch: 8
train:   0%|          | 0/29 [00:03<?, ?it/s]


KeyboardInterrupt: 